# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- import packages used throughout the notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast (even on Colab) because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='ffe2b28ed89637889c04405e5d7d2d98be3df5b6', drake_version='0.27.0', drake_build='release')

from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(
    server_args=['--ngrok_http_tunnel'] if 'google.colab' in sys.modules else [])

from pydrake.common import set_log_level
set_log_level('off');

from underactuated import FindResource

# Simulation: Standing with PID Control

In [ ]:
import numpy as np
from pydrake.all import AddMultibodyPlantSceneGraph, DiagramBuilder, Parser, ConnectMeshcatVisualizer, RigidTransform, Simulator, PidController

def set_home(plant, context):
    hip_roll = .1;
    hip_pitch = 1;
    knee = 1.55;
    plant.GetJointByName("front_right_hip_roll").set_angle(context, -hip_roll)
    plant.GetJointByName("front_right_hip_pitch").set_angle(context, hip_pitch)
    plant.GetJointByName("front_right_knee").set_angle(context, -knee)
    plant.GetJointByName("front_left_hip_roll").set_angle(context, hip_roll)
    plant.GetJointByName("front_left_hip_pitch").set_angle(context, hip_pitch)
    plant.GetJointByName("front_left_knee").set_angle(context, -knee)
    plant.GetJointByName("back_right_hip_roll").set_angle(context, -hip_roll)
    plant.GetJointByName("back_right_hip_pitch").set_angle(context, -hip_pitch)
    plant.GetJointByName("back_right_knee").set_angle(context, knee)
    plant.GetJointByName("back_left_hip_roll").set_angle(context, hip_roll)
    plant.GetJointByName("back_left_hip_pitch").set_angle(context, -hip_pitch)
    plant.GetJointByName("back_left_knee").set_angle(context, knee)
    plant.SetFreeBodyPose(context, plant.GetBodyByName("body"), RigidTransform([0, 0, 0.146]))

def run_pid_control():
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 1e-3)
    parser = Parser(plant)
    parser.AddModelFromFile(FindResource('models/littledog/LittleDog.urdf'))
    parser.AddModelFromFile(FindResource('models/littledog/ground.urdf'))
    plant.Finalize()

    # Add a PD Controller
    kp = 2.0*np.ones(12)
    ki = 0.01*np.ones(12)
    kd = 0.5*np.ones(12)
    kd[-4:] = .16 # use lower gain for the knee joints
    # Select the joint states (and ignore the floating-base states)
    S = np.zeros((24, 37))
    S[:12, 7:19] = np.eye(12)
    S[12:, 25:] = np.eye(12)
    control = builder.AddSystem(PidController(
        kp=kp, ki=ki, kd=kd, 
        state_projection=S, 
        output_projection=plant.MakeActuationMatrix()[6:,:].T))

    builder.Connect(plant.get_state_output_port(), control.get_input_port_estimated_state())
    builder.Connect(control.get_output_port(), plant.get_actuation_input_port())

    visualizer = ConnectMeshcatVisualizer(builder, scene_graph=scene_graph, zmq_url=zmq_url)

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    plant_context = plant.GetMyContextFromRoot(context)
    set_home(plant, plant_context)
    x0 = S @ plant.get_state_output_port().Eval(plant_context)
    control.get_input_port_desired_state().FixValue(control.GetMyContextFromRoot(context), x0)

    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(2.0)

run_pid_control()

# Gait optimization

This is still a work in progress.  I'll finish it soon!

![](https://upload.wikimedia.org/wikipedia/commons/c/cf/Gait_graphs.jpg)

Note: In an old version of Drake, we had a [ComDynamicsFullKinematicsPlanner](https://github.com/RobotLocomotion/drake/blob/last_sha_with_original_matlab/drake/matlab/solvers/trajectoryOptimization/ComDynamicsFullKinematicsPlanner.m).  I reconstructed that functionality explicitly here.


In [ ]:
from pydrake.all import MultibodyPlant, MathematicalProgram, Solve, eq

def gait_optimization(gait = 'walking_trot'):
    plant = MultibodyPlant(1e-3)
    parser = Parser(plant)
    littledog = parser.AddModelFromFile(FindResource('models/littledog/LittleDog.urdf'))
    plant.Finalize()
    context = plant.CreateDefaultContext()

    mu = 1 # rubber on rubber
    total_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(littledog))
    gravity = plant.gravity_field().gravity_vector()
    
    nq = 12
    foot_id = [
        plant.GetFrameByName('front_left_foot_center'),
        plant.GetFrameByName('front_right_foot_center'),
        plant.GetFrameByName('back_left_foot_center'),
        plant.GetFrameByName('back_right_foot_center')]

    # setup gait
    is_laterally_symmetric = False
    check_self_collision = False
    if gait == 'running_trot':
        N = 21
        in_stance = np.zeros((4, N))
        in_stance[1, 3:17] = 1
        in_stance[2, 3:17] = 1
        speed = 0.9
        stride_length = .55
        is_laterally_symmetric = True
    elif gait == 'walking_trot':
        N = 21
        in_stance = np.zeros((4, N))
        in_stance[0, :11] = 1
        in_stance[1, 8:N] = 1
        in_stance[2, 8:N] = 1
        in_stance[3, :11] = 1
        speed = 0.4
        stride_length = .25
        is_laterally_symmetric = True
    elif gait == 'rotary_gallop':
        N = 41
        in_stance = np.zeros((4, N))
        in_stance[0, 7:19] = 1
        in_stance[1, 3:15] = 1
        in_stance[2, 24:35] = 1
        in_stance[3, 26:38] = 1
        speed = 1
        stride_length = .65
        check_self_collision = True
    elif gait == 'bound':
        N = 41
        in_stance = np.zeros((4, N))
        in_stance[0, 6:18] = 1
        in_stance[1, 6:18] = 1
        in_stance[2, 21:32] = 1
        in_stance[3, 21:32] = 1
        speed = 1.2
        stride_length = .55
        check_self_collision = True
    else:
        raise RuntimeError('Unknown gait.')

    
    T = stride_length / speed
    if is_laterally_symmetric:
        T = T / 2.0

    prog = MathematicalProgram()        

    # Time steps    
    h = prog.NewContinuousVariables(N-1, "h")
    prog.AddBoundingBoxConstraint(0.005, 0.1, h) 
    prog.AddLinearConstraint(sum(h) >= .9*T)
    prog.AddLinearConstraint(sum(h) <= 1.1*T)

    # Joint positions and velocities
    q = prog.NewContinuousVariables(plant.num_positions(), N, "q")
    v = prog.NewContinuousVariables(plant.num_velocities(), N, "v")
    # TODO: Use VtoQdot
#    for n in range(N-1):
#        prog.AddConstraint(eq(q[:,n+1], q[:,n] + h[n]*v[:,n]))
    for n in range(N):
        prog.AddBoundingBoxConstraint(plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits(), q[:,n])
        prog.AddBoundingBoxConstraint(plant.GetVelocityLowerLimits(), plant.GetVelocityUpperLimits(), v[:,n])
    # TODO: Add UnitQuaternion constraint

    # Contact forces
    contact_force = [prog.NewContinuousVariables(3, N-1, f"foot{foot}_contact_force") for foot in range(4)]
    for n in range(N-1):
        for foot in range(4):
            # Linear friction cone
            prog.AddLinearConstraint(contact_force[foot][0,n] <= mu*contact_force[foot][2,n])
            prog.AddLinearConstraint(-contact_force[foot][0,n] <= mu*contact_force[foot][2,n])
            prog.AddLinearConstraint(contact_force[foot][1,n] <= mu*contact_force[foot][2,n])
            prog.AddLinearConstraint(-contact_force[foot][1,n] <= mu*contact_force[foot][2,n])
            # normal force >=0, normal_force == 0 if not in_stance
            prog.AddBoundingBoxConstraint(0, in_stance[foot,n]*9.81*total_mass, contact_force[foot][2,n])            

    # Center of mass dynamics
    com = prog.NewContinuousVariables(3, N, "com")
    comdot = prog.NewContinuousVariables(3, N, "comdot")
    comddot = prog.NewContinuousVariables(3, N, "comddot")
#    for n in range(N):
        # TODO: CenterOfMassPosition(q[:,n]) = com[:,n]
    for n in range(N-1):
        prog.AddConstraint(eq(com[:, n+1], com[:,n] + h[n]*comdot[:,n]))
        prog.AddConstraint(eq(comdot[:, n+1], comdot[:,n] + h[n]*comddot[:,n]))
        prog.AddConstraint(eq(total_mass*comddot[:,n+1], comddot[:,n] + h[n]*(sum(contact_force[i][:,n] for i in range(4)) + total_mass*gravity)))

    # Angular momentum dynamics

    result = Solve(prog)
    print(result.is_success())


gait_optimization()